This performs some sanity checks, some with assert statements, some visually which should be gone through manually.

In [ ]:
from pathlib import Path

In [ ]:
import numpy as np

In [ ]:
from tail.analysis.container import Spectra, TheorySpectra, get_idxs

In [ ]:
from dautil.plot import iplot_column_slider

This can be run by both full or not.

In [ ]:
full = False
save = False

In [ ]:
basedir = Path('/scratch/largepatch_new')
path = basedir / 'high_1_0_2.hdf5'
path_cache = basedir / 'high_1_0_2_spectra_final.hdf5'

In [ ]:
bin_width = 100

In [ ]:
%time spectra = Spectra.load(path_cache, bin_width=bin_width, full=full, path_theory_spectra=path)

In [ ]:
%time spectra = spectra.slicing_l(600, 3000, ascontiguousarray=True)

In [ ]:
spectra.b_range

In [ ]:
spectra.subtracting_leakage()

# Leakage, signal, theory

In [ ]:
leakage = spectra.leakage

In [ ]:
leakage_max = np.abs(leakage).max()
leakage_max

Leakage after leakage subtraction should be tiny

In [ ]:
assert leakage_max <= 1e-16

In [ ]:
idxs_cross = get_idxs(spectra.spectra, ['TE', 'TB', 'EB'])
maxs = []
for map_case in spectra.map_cases:
    Nl = getattr(spectra, map_case)
    for idx in idxs_cross:
        temp = np.abs(Nl[idx, :, :, :, 1, :]).max()
        print(map_case, spectra.spectra[idx], temp)
        maxs.append(temp)

In [ ]:
signal = spectra.signal
theory = spectra.theory

By definition of the filter transfer function, the BPWF-theory should equals to the constructed singal (using no noise simulation) up to floating point error.

In [ ]:
df_ratio = spectra.to_frame_4d(signal / theory - 1.)

Only TE is non-trivial here. Ideally it is very close to 0. Error on filter transfer function model shows up here.

In [ ]:
iplot_column_slider(df_ratio.loc[['TE']].T)

In [ ]:
spectra_idxs = get_idxs(spectra.spectra, ('TT', 'EE', 'BB'))
print(f'Requiring auto-spectra of theory matching signal: {spectra.spectra[spectra_idxs]}')
_, n_null_split, n_sub_split, _ = signal.shape
np.testing.assert_allclose(theory[spectra_idxs], signal[spectra_idxs])

# chi-distribution

In [ ]:
err_mc = spectra.err_mc

In [ ]:
err_rel = (spectra.realmaps[:, :, :, :, 0, 0] - signal) / err_mc

In [ ]:
df_err_rel = spectra.to_frame_4d(err_rel)

In [ ]:
df_chi_mean = df_err_rel.std(axis=1).unstack(level=0)

These no. should be close to 1

In [ ]:
df_chi_mean

In [ ]:
if save:
    df_chi_mean.to_csv('chi_mean.csv', float_format='%.3g')

# Chi-sq

In [ ]:
from scipy.stats import chi2

In [ ]:
dist = chi2(err_rel.shape[-1])

In [ ]:
err_rel.shape

In [ ]:
df_chi_sq = df_err_rel.apply(np.square).sum(axis=1).unstack(level=0)

In [ ]:
df_p = df_chi_sq.apply(lambda x: dist.cdf(x))

In [ ]:
df_p

In [ ]:
if save:
    df_p.to_csv('pte_iid.csv', float_format='%.3g')